In [2]:
import pandas as pd
from sqlalchemy import create_engine
price_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_price_yf.db')
ticker_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_list_with_name.db')
ticker_df = pd.read_sql('TOTAL', con=ticker_engine)
ticker_list = ticker_df.Symbol.to_list()

In [4]:
import tushare as ts
pro = ts.pro_api(token='28ce867873a056747209a2ef0ff53d98ed850903fd954417ac69553b')

#获取富时中国50指数
df = pro.index_global(ts_code='SPX', start_date='20000201', end_date='20220913')

In [7]:
downtrend_list =[['2022-01-03', '2022-06-21'],
                ['2020-02-20', '2022-03-23'],
                ['2018-10-03', '2018-12-26']]
start_date = downtrend_list[0][0]
end_date = downtrend_list[0][1]
start_index = df[df.trade_date == start_date.replace('-','')].index.values[0]
end_index = df[df.trade_date == end_date.replace('-','')].index.values[0]
last_count = abs(start_index - end_index)
last_count

115

In [11]:
ticker = 'AAPL'
ticker = 'BLAT'
ticker = 'VIEW'
res = pd.DataFrame()
ticker_df = pd.read_sql('{}'.format(ticker),con=price_engine)
ticker_df.rename(columns={'hgih':'high'},inplace=True)
# 初始价格
start_price = ticker_df[ticker_df.datetime == start_date].close.values[0]
# 初始价格时的index
tstart_index = ticker_df[ticker_df.datetime == start_date].index.values[0]
# 下降中的最低价
lowest_price = min(ticker_df[tstart_index:tstart_index+int(last_count*1.33)].low)
# 最低价时的index
lowest_index = ticker_df[ticker_df.low == lowest_price].index.values[0]
# 下降率
decline_rate = round((start_price - lowest_price) / start_price, 3)
# 下降时间
tlast_count = abs(lowest_index - tstart_index)
# 更新df的范围，起始点为最低价
ticker_df = ticker_df[lowest_index:]

# 获取到达原价75%时的index和价格
if ticker_df[ticker_df.close >= start_price * 0.75].empty != True:
    rev_h_index = ticker_df[ticker_df.close >= start_price * 0.75].index.values[0]
    rev_h_price = ticker_df[ticker_df.close >= start_price * 0.75].close.values[0]
else:
    rev_h_index = 9999
    rev_h_price = 9999
# 获取到达最低价125%时的index
if ticker_df[ticker_df.close >= lowest_price * 1.25].empty != True:
    rev_l_index = ticker_df[ticker_df.close >= lowest_price * 1.25].index.values[0]
    rev_l_price = ticker_df[ticker_df.close >= lowest_price * 1.25].close.values[0]
else: 
    rev_l_index = 9999
    rev_l_price = 9999

# 进场点（用于计算收益，取小的index）
if rev_h_index == rev_l_index == 9999:
    print('can not continue')
elif rev_h_index == 9999:
    rev_h_count = 9999
    rev_l_count = abs(rev_l_index - lowest_index)
    enter_point = rev_l_count
    enter_price = rev_l_price
elif rev_l_index == 9999:
    rev_l_count = 9999
    rev_h_count = abs(rev_h_index - lowest_index)
    enter_point = rev_h_count
    enter_price = rev_h_price
else:
    # 分别计算到达的时间
    rev_h_count = abs(rev_h_index - lowest_index)
    rev_l_count = abs(rev_l_index - lowest_index)
    if rev_l_count < rev_h_count:
        enter_point = rev_l_count
        enter_price = rev_l_price
    else:
        enter_point = rev_h_count
        enter_price = rev_h_price
# 更新df范围，起始点变为进场点
ticker_df = ticker_df[enter_point:]
max_30 = max(ticker_df[:int(30/7*5)].close)
max_60 = max(ticker_df[:int(60/7*5)].close)
max_90 = max(ticker_df[:int(90/7*5)].close)
max_180 = max(ticker_df[:int(180/7*5)].close)
profit_30 = (max_30 - enter_price) / enter_price
profit_60 = (max_60 - enter_price) / enter_price
profit_90 = (max_90 - enter_price) / enter_price
profit_180 = (max_180 - enter_price) / enter_price
temp = pd.DataFrame([[ticker,decline_rate,tlast_count,rev_h_count, rev_l_count, profit_30,profit_60,profit_90,profit_180]],columns=['ticker','dec_rate','dec_time','rec_h_time','rec_l_time','pft_30','pft_60','pft_90','pft_180'])
res = pd.concat([res,temp],ignore_index=True)
res

,ticker,dec_rate,dec_time,rec_h_time,rec_l_time,pft_30,pft_60,pft_90,pft_180
0,VIEW,0.903,89,9606,0,3.96124,3.96124,3.96124,3.96124


In [8]:
test = pd.DataFrame()
res = pd.concat([res, test],ignore_index=True)
res

,ticker,dec_rate,dec_time,rec_h_time,rec_l_time,pft_30,pft_60,pft_90,pft_180
0,AAPL,0.388,62,22,45,0.009588,0.123493,0.215696,0.223103


In [104]:
temp

,ticker,dec_rate,dec_time,rec_l_time,rec_h_time,pft_30,pft_60,pft_90,pft_180


In [101]:
decline_rate,tlast_count

(0.388, 62)

In [102]:
rev_h_count,rev_l_count

(84, 107)

In [100]:
profit_30,profit_60,profit_90,profit_180,

(0.16596439508661723,
 0.16596439508661723,
 0.21317253233178732,
 0.32449102752452325)

ValueError: max() arg is an empty sequence

In [69]:
profit_30,profit_60,profit_90,profit_180

(-1376.3594973827155,
 -1376.3594973827155,
 -1376.3594973827155,
 -1376.3594973827155)

In [63]:
recover_l25_index, rev_h_count

(10325, 256)

In [65]:
rev_l_count, rev_h_count

(27, 256)

In [28]:
aapl[aapl.datetime == '2018-12-26']

,open,hgih,low,close,Adj Close,volume,datetime
9591,37.075001,39.307499,36.68,39.2925,38.039845,234330000,2018-12-26
